# Mini Project 2: IMDB Sentiment Analysis

February 22, 2019

Akshal Aniche, Jacob Sanz-Robinson, Raphael Hotter

COMP 551

## Imports

In [15]:
import numpy as np
from sklearn.model_selection import cross_val_score

## Load the data

In [16]:
import os
pos_train_files = os.listdir('data/train/pos/')
neg_train_files = os.listdir('data/train/neg/')
test_files = os.listdir('data/test/')

pos_words = 'opinion-lexicon-English/positive-words.txt'
neg_words = 'opinion-lexicon-English/negative-words.txt'


# Remove .DS_Store files
while '.DS_Store' in pos_train_files:
  pos_train_files.remove('.DS_Store')
while '.DS_Store' in neg_train_files:
  neg_train_files.remove('.DS_Store')
while '.DS_Store' in test_files:
  test_files.remove('.DS_Store')

In [17]:
# Reads data into 2 arrays
train_text, test_raw = [], []

for file in pos_train_files:
  with open('data/train/pos/{}'.format(file), 'r') as f:
    train_text.append(f.read())

for file in neg_train_files:
  with open('data/train/neg/{}'.format(file), 'r') as f:
    train_text.append(f.read())

for file in test_files:
  with open('data/test/{}'.format(file), 'r') as f:
    test_raw.append(f.read())

In [18]:
# Training set Y vector
pos_goal = np.ones((12500))
neg_goal = np.zeros((12500))
Y_train = np.append(pos_goal, neg_goal, axis = 0)

### Read lexicon of sentiment words

In [19]:
sent_words = []

with open(pos_words, 'r', encoding="latin-1") as f:
    for line in f:
        if (line[0] != ';'):
            sent_words.append(line.strip())

with open(neg_words, 'r', encoding="latin-1") as f:o
    for line in f:
        if (line[0] != ';'):
            sent_words.append(line.strip())

            
print(sent_words)

['', 'a+', 'abound', 'abounds', 'abundance', 'abundant', 'accessable', 'accessible', 'acclaim', 'acclaimed', 'acclamation', 'accolade', 'accolades', 'accommodative', 'accomodative', 'accomplish', 'accomplished', 'accomplishment', 'accomplishments', 'accurate', 'accurately', 'achievable', 'achievement', 'achievements', 'achievible', 'acumen', 'adaptable', 'adaptive', 'adequate', 'adjustable', 'admirable', 'admirably', 'admiration', 'admire', 'admirer', 'admiring', 'admiringly', 'adorable', 'adore', 'adored', 'adorer', 'adoring', 'adoringly', 'adroit', 'adroitly', 'adulate', 'adulation', 'adulatory', 'advanced', 'advantage', 'advantageous', 'advantageously', 'advantages', 'adventuresome', 'adventurous', 'advocate', 'advocated', 'advocates', 'affability', 'affable', 'affably', 'affectation', 'affection', 'affectionate', 'affinity', 'affirm', 'affirmation', 'affirmative', 'affluence', 'affluent', 'afford', 'affordable', 'affordably', 'afordable', 'agile', 'agilely', 'agility', 'agreeable',

## Data processer preparation

In [20]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import Normalizer

bin_vec = CountVectorizer(binary=True)
tfidf_vec = TfidfVectorizer()
bigram_vec = CountVectorizer(ngram_range=(1,2))
sentc_vec = CountVectorizer(binary=True)
sentt_vec = TfidfVectorizer()

## Text processing

### Binary representation

In [21]:
# Tokenize and build the vocabulary
bin_vec.fit(train_text)

# Create the feautre matrices 
train_bin = bin_vec.transform(train_text)
test_bin = bin_vec.transform(test_raw)

## Binary representation, only sentiment lexicon words

In [22]:
# Tokenize and build vocabulary 
sentc_vec.fit(sent_words)

# Create the feautre matrices 
train_s_bin = sentc_vec.transform(train_text)
test_s_bin = sentc_vec.transform(test_raw)

### Monograms and Bigrams 

In [23]:
# Tokenize and build the vocabulary
bigram_vec.fit(train_text)

# Create the feautre matrices 
train_bigram = bigram_vec.transform(train_text)
test_bigram = bigram_vec.transform(test_raw)

### Tf-Idf 

In [24]:
# Tokenize and build the vocabulary
tfidf_vec.fit(train_text)

# Create the feautre matrices 
train_tfidf = tfidf_vec.transform(train_text)
test_tfidf = tfidf_vec.transform(test_raw)

### Tf-Idf with only sentiment lexicon, Normalized

In [25]:
# Tokenize and build the vocabulary
sentt_vec.fit(sent_words)

# Create the feautre matrices 
train_s_tfidf = sentt_vec.transform(train_text)
test_s_tfidf = sentt_vec.transform(test_raw)

normalizer_tran = Normalizer().fit(X=train_s_tfidf)
X_train_sn = normalizer_tran.transform(train_s_tfidf)
X_test_sn = normalizer_tran.transform(test_s_tfidf)

### Normalized Tf-Idf

In [26]:
normalizer_tranformer = Normalizer().fit(X=train_tfidf)
X_train_normalized = normalizer_tranformer.transform(train_tfidf)
X_test_normalized = normalizer_tranformer.transform(test_tfidf)

## Bernoulli Naive Bayes model from scratch

In [6]:
numPos = len(pos_train_files)
numNeg = len(neg_train_files)
numTot = len(train_text)
#print(numPos, numNeg, numTot)

#Prior probability calculation
probPos = float(numPos)/numTot
probNeg = float(numNeg)/numTot
#print(probPos, probNeg)

In [7]:
theta_j1 = [0] * len(bin_vec.get_feature_names()) #instantiate array of 0's. One cell per each word in bag of words.
exampleNum = 0
while exampleNum < 500:#12499: # for each review
    currentWordNum = 0 #index to keep track of current word's index
    for wordOccurs in train_bin[exampleNum].toarray().sum(axis=0):
        if wordOccurs != 0: #if a word occurs, then we add to its count in the array
            theta_j1[currentWordNum] = theta_j1[currentWordNum] + 1
        currentWordNum = currentWordNum + 1
    exampleNum = exampleNum + 1
    #if exampleNum%100 == 0:
        #print exampleNum

theta_j1_np = np.array(theta_j1)
theta_j1_final = theta_j1_np/float(numPos)

NotFittedError: CountVectorizer - Vocabulary wasn't fitted.

In [ ]:
theta_j0 = [0] * len(bin_vec.get_feature_names()) #same as above for theta_j1
exampleNum = 12500
while exampleNum < 13200:#24999: 
    currentWordNum = 0 
    for wordOccurs in train_bin[exampleNum].toarray().sum(axis=0):
        if wordOccurs != 0: 
            theta_j0[currentWordNum] = theta_j0[currentWordNum] + 1
        currentWordNum = currentWordNum + 1
    exampleNum = exampleNum + 1
    #if exampleNum%100 == 0:
        #print exampleNum

theta_j0_np = np.array(theta_j0)
theta_j0_final = theta_j0_np/float(numNeg)

## Models

### Naive Bayes (to make sure it worked)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
bayes_clf = MultinomialNB()

#### Cross Validation

In [ ]:
scores = cross_val_score(bayes_clf, X_train_normalized, Y_train, cv=4)
mean = scores.mean()
print("Scores: {}".format(scores))
print("Scores Mean: {}".format(mean))

### Logistic Regression

In [30]:
from sklearn import linear_model
regr_clf = linear_model.LogisticRegression(solver='lbfgs')

#### Cross Validation

In [31]:
scores = cross_val_score(regr_clf, X_train_normalized, Y_train, cv=4)
mean = scores.mean()
print("Scores: {}".format(scores))
print("Scores Mean: {}".format(mean))

Scores: [0.86464 0.86112 0.8592  0.868  ]
Scores Mean: 0.8632399999999999


In [32]:
scores = cross_val_score(regr_clf, X_train_sn, Y_train, cv=4)
mean = scores.mean()
print("Scores: {}".format(scores))
print("Scores Mean: {}".format(mean))

Scores: [0.82448 0.83568 0.82384 0.82544]
Scores Mean: 0.82736


### Decision Trees

In [ ]:
from sklearn import tree
dec_clf = tree.DecisionTreeClassifier()

In [ ]:
scores = cross_val_score(dec_clf, X_train_normalized, Y_train, cv=4)
mean = scores.mean()
print("Scores: {}".format(scores))
print("Scores Mean: {}".format(mean))

I wonder if we can play with the parameters more 

### Support Vector Machines

In [27]:
from sklearn import svm
svm_clf = svm.LinearSVC()

In [28]:
scores = cross_val_score(svm_clf, X_train_normalized, Y_train, cv=4)
mean = scores.mean()
print("Scores: {}".format(scores))
print("Scores Mean: {}".format(mean))

Scores: [0.84704 0.82496 0.84832 0.84848]
Scores Mean: 0.8422


In [29]:
scores = cross_val_score(svm_clf, X_train_sn, Y_train, cv=4)
mean = scores.mean()
print("Scores: {}".format(scores))
print("Scores Mean: {}".format(mean))

Scores: [0.8472  0.84976 0.8448  0.84768]
Scores Mean: 0.84736


# Cross Validation

To test standard Naive Bayes:

In [ ]:
# Define train(), evaluate() functions
def k_cross_validate(X_train, Y_train, k):
    full_matrix = np.concatenate((X_train, Y_train), axis=1)
    np.random.shuffle(full_matrix) # shuffles rows
    X_train = full_matrix[:,:-1]
    Y_train = full_matrix[:,-1]
    groups = []
    num_rows = X_train.shape[0]
    spacing = int(num_rows/k)
    
    # split into groups
    for i in range(k-1):
        X_mini_train = np.concatenate((X_train[i*spacing:(i+1)spacing, :]), axis=0)
              
              
            groups.append((X_train[i*spacing:(i+1)spacing, :], X_train[i*spacing:(i+1)spacing, :]))
    groups.append((X_train[(k-1)*spacing, :], Y_train[(k-1)*spacing, :]))
        